In [ ]:
%run /Workspace/Projects/Experimentation/aaml-experimentation-coe/exp_coe_utils


In [ ]:
# =============================================================================
# MARKDOWN TEST TABLE CREATION
# =============================================================================
# Creates a test version of combined_txns table with 3 markdown approaches:
#   1. TOTAL_MARKDOWN = REVENUE - NET_SALES (Derived)
#   2. MKDN_AMT (Justin's approach - native column)
#   3. COMBINED_MKDN = MKDN_AMT + WOD + POD (Vishwas's approach)
# =============================================================================

# Date parameters - adjust as needed
start_date = "CURRENT_DATE()-90"
end_date = "CURRENT_DATE()"

# Data source constants
TXN_FACTS = 'gcp-abs-udco-bqvw-prod-prj-01.udco_ds_retl.txn_facts'
TXN_HDR_COMBINED = 'gcp-abs-udco-bqvw-prod-prj-01.udco_ds_retl.TXN_HDR_COMBINED'
LU_DAY_MERGE = 'gcp-abs-udco-bqvw-prod-prj-01.udco_ds_edw.LU_DAY_MERGE'
SMV_RETAIL_CUSTOMER_LOYALTY_PROGRAM_HOUSEHOLD = 'gcp-abs-udco-bqvw-prod-prj-01.udco_ds_cust.SMV_RETAIL_CUSTOMER_LOYALTY_PROGRAM_HOUSEHOLD'

# Output table (test version)
OUTPUT_TABLE = "db_work.EXP_COE_COMBINED_TXNS_GCP_markdown_test"

print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")
print(f"Output Table: {OUTPUT_TABLE}")


In [ ]:
# Combined Transactions Query with ALL 3 Markdown Approaches
combined_txns_markdown_query = (
f"""
WITH filtered_tf AS (
  SELECT
    TXN_ID,
    TXN_DTE,
    CARD_NBR,
    SUM(GROSS_AMT) AS REVENUE,
    SUM(NET_AMT+MKDN_WOD_ALLOC_AMT+MKDN_POD_ALLOC_AMT) AS NET_SALES,
    SUM(ITEM_QTY) AS ITEMS,
    -- Justin's approach: Raw MKDN_AMT
    SUM(MKDN_AMT) AS MKDN_AMT,
    -- Vishwas's approach: Combined markdown columns
    SUM(MKDN_AMT + MKDN_WOD_ALLOC_AMT + MKDN_POD_ALLOC_AMT) AS COMBINED_MKDN
  FROM {TXN_FACTS}
  WHERE
    TXN_DTE >= {start_date} 
    AND TXN_DTE < {end_date}
    AND MISC_ITEM_QTY = 0
    AND DEPOSIT_ITEM_QTY = 0
    AND REV_DTL_SUBTYPE_ID IN (0, 6, 7)
  GROUP BY TXN_ID, TXN_DTE, CARD_NBR
)
SELECT
  smv.HOUSEHOLD_ID,
  tf.TXN_DTE,
  tf.TXN_ID,
  CASE
    WHEN f.REGISTER_NBR IN (99, 104, 173, 174, 999) THEN 'ECOMM'
    WHEN f.REGISTER_NBR IN (
      1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
      11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
      49, 50, 51, 52, 53, 54, 93, 94, 95, 96, 97, 98,
      116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
      151, 152, 153, 154, 175, 176, 177, 178, 179, 180,
      181, 182, 195, 196, 197, 198
    ) THEN 'STORE'
    ELSE NULL
  END AS TXN_LOCATION,
  tf.REVENUE,
  tf.NET_SALES,
  tf.ITEMS,
  f.TENDER_AMT_FOODSTAMPS + f.TENDER_AMT_EBT AS SNAP_TENDER,
  -- Derived (R-NS): Our calculated markdown
  (tf.REVENUE - tf.NET_SALES) AS TOTAL_MARKDOWN,
  -- Justin's approach
  tf.MKDN_AMT,
  -- Vishwas's approach
  tf.COMBINED_MKDN
FROM filtered_tf AS tf
JOIN {TXN_HDR_COMBINED} AS f
  ON tf.TXN_ID = f.TXN_ID AND tf.TXN_DTE = f.TXN_DTE
JOIN {LU_DAY_MERGE} AS b
  ON CAST(f.TXN_DTE AS DATE) = b.D_DATE
JOIN (
  SELECT DISTINCT HOUSEHOLD_ID, LOYALTY_PROGRAM_CARD_NBR
  FROM {SMV_RETAIL_CUSTOMER_LOYALTY_PROGRAM_HOUSEHOLD}
) AS smv
  ON SAFE_CAST(tf.CARD_NBR AS BIGNUMERIC) = SAFE_CAST(smv.LOYALTY_PROGRAM_CARD_NBR AS BIGNUMERIC)
WHERE
  f.TXN_HDR_SRC_CD = 0
  AND f.REGISTER_NBR IN (
    99, 104, 173, 174, 999,
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
    11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
    49, 50, 51, 52, 53, 54, 93, 94, 95, 96, 97, 98,
    116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
    151, 152, 153, 154, 175, 176, 177, 178, 179, 180,
    181, 182, 195, 196, 197, 198
  )
"""
)

print("Query created with 3 markdown approaches:")
print("  1. TOTAL_MARKDOWN = REVENUE - NET_SALES (Derived)")
print("  2. MKDN_AMT (Justin's)")
print("  3. COMBINED_MKDN (Vishwas's)")
print()
print(combined_txns_markdown_query)


In [ ]:
# Execute query and save to test table
print(f"Executing query and saving to: {OUTPUT_TABLE}")
print("This may take a few minutes...")

combined_txns_sp = bc.read_gcp_table(combined_txns_markdown_query)
combined_txns_sp.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(OUTPUT_TABLE)

print(f"\n✅ Table saved: {OUTPUT_TABLE}")
print(f"Row count: {combined_txns_sp.count():,}")


In [ ]:
# Verify the table - show schema and sample
print("=" * 60)
print("TABLE SCHEMA")
print("=" * 60)
combined_txns_sp.printSchema()

print("\n" + "=" * 60)
print("SAMPLE DATA (5 rows)")
print("=" * 60)
display(combined_txns_sp.limit(5))

print("\n" + "=" * 60)
print("MARKDOWN COLUMNS SUMMARY")
print("=" * 60)
print("Columns available for comparison:")
print("  • TOTAL_MARKDOWN  - Derived (R-NS): REVENUE - NET_SALES")
print("  • MKDN_AMT        - Justin's: Raw markdown from TXN_FACTS")
print("  • COMBINED_MKDN   - Vishwas's: MKDN_AMT + WOD + POD")


# Markdown Fix - Testing Notebook

This notebook creates a **TEST version** of the Combined Transactions table with the new `TOTAL_MARKDOWN` column.

**Output Table:** `db_work.EXP_COE_COMBINED_TXNS_GCP_markdown_test`

**Changes Made:**
- Added `(tf.REVENUE - tf.NET_SALES) AS TOTAL_MARKDOWN` to the SELECT clause

**Next Steps:**
1. Run this notebook to create the test table
2. Update SAFE notebook to point to `_markdown_test` table
3. Compare experiment results with old vs new metrics
4. If validated, apply changes to main `metric_workflow.ipynb`


In [ ]:
# Imports
%run /Workspace/Projects/Experimentation/aaml-experimentation-coe/exp_coe_utils


In [ ]:
# Data Source Constants (GCP BigQuery)
TXN_FACTS = 'gcp-abs-udco-bqvw-prod-prj-01.udco_ds_retl.txn_facts'
TXN_HDR_COMBINED = 'gcp-abs-udco-bqvw-prod-prj-01.udco_ds_retl.TXN_HDR_COMBINED'
LU_DAY_MERGE = 'gcp-abs-udco-bqvw-prod-prj-01.udco_ds_edw.LU_DAY_MERGE'
SMV_RETAIL_CUSTOMER_LOYALTY_PROGRAM_HOUSEHOLD = 'gcp-abs-udco-bqvw-prod-prj-01.udco_ds_cust.SMV_RETAIL_CUSTOMER_LOYALTY_PROGRAM_HOUSEHOLD'

# TEST Output Table (with _markdown_test suffix)
COMBINED_TXNS_TEST = "db_work.EXP_COE_COMBINED_TXNS_GCP_markdown_test"


In [ ]:
# Date Range Configuration
# Uses last 60 days by default (same as standard metric pipeline)
from datetime import datetime, timedelta

end_dt = datetime.now()
start_dt = end_dt - timedelta(days=60)

start_date = f"DATE('{start_dt.strftime('%Y-%m-%d')}')"
end_date = f"DATE('{end_dt.strftime('%Y-%m-%d')}')"

print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")


In [ ]:
# Combined Transactions Query - WITH TOTAL_MARKDOWN (NEW COLUMN)
combined_txns_query = f"""
WITH filtered_tf AS (
  SELECT
    TXN_ID,
    TXN_DTE,
    CARD_NBR,
    SUM(GROSS_AMT) AS REVENUE,
    SUM(NET_AMT+MKDN_WOD_ALLOC_AMT+MKDN_POD_ALLOC_AMT) AS NET_SALES,
    SUM(ITEM_QTY) AS ITEMS
  FROM {TXN_FACTS}
  WHERE
    TXN_DTE >= {start_date} 
    AND TXN_DTE < {end_date}
    AND MISC_ITEM_QTY = 0
    AND DEPOSIT_ITEM_QTY = 0
    AND REV_DTL_SUBTYPE_ID IN (0, 6, 7)
  GROUP BY TXN_ID, TXN_DTE, CARD_NBR
)
SELECT
  smv.HOUSEHOLD_ID,
  tf.TXN_DTE,
  tf.TXN_ID,
  CASE
    WHEN f.REGISTER_NBR IN (99, 104, 173, 174, 999) THEN 'ECOMM'
    WHEN f.REGISTER_NBR IN (
      1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
      11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
      49, 50, 51, 52, 53, 54, 93, 94, 95, 96, 97, 98,
      116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
      151, 152, 153, 154, 175, 176, 177, 178, 179, 180,
      181, 182, 195, 196, 197, 198
    ) THEN 'STORE'
    ELSE NULL
  END AS TXN_LOCATION,
  tf.REVENUE,
  tf.NET_SALES,
  (tf.REVENUE - tf.NET_SALES) AS TOTAL_MARKDOWN,
  tf.ITEMS,
  f.TENDER_AMT_FOODSTAMPS + f.TENDER_AMT_EBT AS SNAP_TENDER
FROM filtered_tf AS tf
JOIN {TXN_HDR_COMBINED} AS f
  ON tf.TXN_ID = f.TXN_ID AND tf.TXN_DTE = f.TXN_DTE
JOIN {LU_DAY_MERGE} AS b
  ON CAST(f.TXN_DTE AS DATE) = b.D_DATE
JOIN (
  SELECT DISTINCT HOUSEHOLD_ID, LOYALTY_PROGRAM_CARD_NBR
  FROM {SMV_RETAIL_CUSTOMER_LOYALTY_PROGRAM_HOUSEHOLD}
) AS smv
  ON SAFE_CAST(tf.CARD_NBR AS BIGNUMERIC) = SAFE_CAST(smv.LOYALTY_PROGRAM_CARD_NBR AS BIGNUMERIC)
WHERE
  f.TXN_HDR_SRC_CD = 0
  AND f.REGISTER_NBR IN (
    99, 104, 173, 174, 999,
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
    11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
    49, 50, 51, 52, 53, 54, 93, 94, 95, 96, 97, 98,
    116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
    151, 152, 153, 154, 175, 176, 177, 178, 179, 180,
    181, 182, 195, 196, 197, 198
  )
"""

print("Query built successfully!")
print(f"Output table: {COMBINED_TXNS_TEST}")


In [ ]:
# Execute Query and Save to TEST Table
print(f"Executing query and saving to: {COMBINED_TXNS_TEST}")

combined_txns_sp = bc.read_gcp_table(combined_txns_query)
combined_txns_sp.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(COMBINED_TXNS_TEST)

print(f"SUCCESS! Table saved: {COMBINED_TXNS_TEST}")


---
## Validation

Run the cell below to verify the test table was created correctly and the math balances.


In [ ]:
# Validation: Check table schema and math
validation_df = spark.sql(f"""
SELECT 
    'VALIDATION' AS CHECK_TYPE,
    COUNT(*) AS ROW_COUNT,
    SUM(REVENUE) AS TOTAL_REVENUE,
    SUM(NET_SALES) AS TOTAL_NET_SALES,
    SUM(TOTAL_MARKDOWN) AS TOTAL_MARKDOWN_SUM,
    SUM(REVENUE) - SUM(NET_SALES) AS EXPECTED_MARKDOWN,
    SUM(REVENUE) - SUM(NET_SALES) - SUM(TOTAL_MARKDOWN) AS DIFFERENCE_SHOULD_BE_ZERO
FROM {COMBINED_TXNS_TEST}
""")

display(validation_df)


In [ ]:
# Show table schema (should include TOTAL_MARKDOWN column)
spark.sql(f"DESCRIBE {COMBINED_TXNS_TEST}").display()


---
## Compare with Original Table

Run the cells below to compare the test table with the original production table.


In [ ]:
# Compare TEST table vs ORIGINAL table
ORIGINAL_TABLE = "db_work.EXP_COE_COMBINED_TXNS_GCP"

comparison_df = spark.sql(f"""
SELECT 
    'ORIGINAL' AS TABLE_TYPE,
    COUNT(*) AS ROW_COUNT,
    SUM(REVENUE) AS TOTAL_REVENUE,
    SUM(NET_SALES) AS TOTAL_NET_SALES
FROM {ORIGINAL_TABLE}
UNION ALL
SELECT 
    'TEST (with TOTAL_MARKDOWN)' AS TABLE_TYPE,
    COUNT(*) AS ROW_COUNT,
    SUM(REVENUE) AS TOTAL_REVENUE,
    SUM(NET_SALES) AS TOTAL_NET_SALES
FROM {COMBINED_TXNS_TEST}
""")

display(comparison_df)


---
## Using in SAFE Notebook

To test with SAFE, update the table reference in SAFE notebook:

**Original:**
```python
db_work.EXP_COE_COMBINED_TXNS_GCP
```

**Test:**
```python
db_work.EXP_COE_COMBINED_TXNS_GCP_markdown_test
```

The test table has the same schema as the original PLUS the new `TOTAL_MARKDOWN` column.
